### Tweepy Documentation
http://docs.tweepy.org/en/v3.5.0/api.html

### Authentication

In [11]:
import tweepy
import os

API_Key = os.environ.get('Twitter_API_Key')
API_Secret = os.environ.get('Twitter_API_Secret')
Access_Token = os.environ.get('Twitter_Access_Key')
Access_Secret = os.environ.get('Twitter_Access_Secret')

auth = tweepy.OAuthHandler(API_Key,API_Secret)
auth.set_access_token(Access_Token, Access_Secret)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True, compression=True)

### Getting a user via id and vice versa

http://docs.tweepy.org/en/v3.5.0/api.html?highlight=get_user#API.get_user

https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/user-object.html -- information on User Objects

In [12]:
u=api.get_user('Tolomeo_R')
print(u.id)

4442463679


In [13]:
u = api.get_user(u.id)
print(u.screen_name)
print(u.profile_image_url_https)
print('https://www.twitter.com/{}'.format(u.screen_name))

Tolomeo_R
https://pbs.twimg.com/profile_images/952241962261737473/J952D6l-_normal.jpg
https://www.twitter.com/Tolomeo_R


### Parsing a user's followers using paging

Will hit the rate limit using api.followers

In [28]:
user_name = 'Tolomeo_R' # this is their @handle

ids = []
i = 1
for page in tweepy.Cursor(api.followers_ids, user_name).pages():
    ids.extend(page)
    print('Page {} length:{}'.format(str(i)+"'s",len(page)))
    i += 1
print(len(ids))

C:\Users\kevin\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  


ArgSpec(args=['self', 'method'], varargs='args', keywords='kargs', defaults=None)
Page 1's length:5000


Rate limit reached. Sleeping for: 603


Page 2's length:234
5234


### Checking the rate limit

In [15]:
data = api.rate_limit_status()

In [16]:
data['resources']['followers']

{'/followers/ids': {'limit': 15, 'remaining': 11, 'reset': 1568837757},
 '/followers/list': {'limit': 15, 'remaining': 0, 'reset': 1568837677}}

### Time until reset

In [17]:
reset = data['resources']['followers']['/followers/ids']['reset']

In [18]:
import time

time.ctime(reset)

'Wed Sep 18 13:15:57 2019'

In [19]:
reset - time.time()

892.1358139514923

In [20]:
data = api.rate_limit_status()
resources = data['resources']

for resource in resources:
    for value in resources[resource]:
        limit = resources[resource][value]['limit']
        remaining = resources[resource][value]['remaining']
        
        if limit != remaining:
            reset = resources[resource][value]['reset']
            reset - time.time()
            print("{} out of {} {} requests remaining. Resets {} seconds from now.".format(remaining,limit,value,reset-time.time()))

175 out of 180 /application/rate_limit_status requests remaining. Resets 141.56113815307617 seconds from now.
894 out of 900 /users/show/:id requests remaining. Resets 810.5591418743134 seconds from now.
11 out of 15 /followers/ids requests remaining. Resets 891.5581550598145 seconds from now.
0 out of 15 /followers/list requests remaining. Resets 811.5581550598145 seconds from now.
